In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow-addons
!pip install -q tensorflow-io

In [3]:
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import scipy.io as sio
import pandas as pd
import time
import cv2
import tensorflow_addons as tfa
from skimage.transform import resize
from IPython.display import clear_output
import tensorflow_io as tfio

In [4]:
def one_hot(vec):
  items=np.sort(pd.unique(vec))
  n_class=np.shape(items)[0]
  zeros=np.zeros((vec.size, n_class))
  for n,i in enumerate(items):
    rows=np.where(vec==i)[0]
    zeros[rows,n]=1
  return zeros.astype('float32'), n_class

In [5]:
def get_index(vec_lab, seed=1996, split_train=0.6, validation=False):
  train=[]
  valid=[]
  test =[]

  spv=(1-split_train)/2.
  for i in range(2):
    ind_clss=np.where(vec_lab[:,i]==1)[0]
    sz=len(ind_clss)
    np.random.seed(seed)
    ramd=np.random.choice(sz, sz, replace=False)
    train=np.append(train, ind_clss[ramd[:int(sz*split_train)]])
    valid=np.append(valid, ind_clss[ramd[int(sz*split_train):int(sz*(split_train+spv))]])
    test =np.append(test, ind_clss[ramd[int(sz*(split_train+spv)):]])
  train=train[np.random.choice(len(train), len(train), replace=False)].astype('int')
  valid=valid[np.random.choice(len(valid), len(valid), replace=False)].astype('int')
  test= test[np.random.choice(len(test), len(test), replace=False)].astype('int')
  if validation:
    return train, valid, test
  else:
    return np.concatenate((train,valid)), test

### Data augmentation functions

In [6]:
def none_p(data, labx):
  return data, labx

In [7]:
def aug_translation(data, labx):
  sz=np.shape(data)
  new=np.zeros((3,*sz))
  tx=int(sz[1]/4)
  ty=int(sz[2]/4)  
  new[0,:,tx:,ty:,:]=data[:,:-tx,:-ty,:]
  new[1,:,:,ty:,:]=data[:,:,:-ty,:]
  new[2,:,tx:,:,:]=data[:,:-tx,:,:]
  images=tf.concat([data,
                    new[0],
                    new[1],
                    new[2]], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [8]:
def aug_rotation(data, labx):
  images=tf.concat([data,
                  tfa.image.rotate(data, angles=np.pi/2),
                  tfa.image.rotate(data, angles=np.pi),
                  tfa.image.rotate(data, angles=np.pi*3/2)], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [9]:
def aug_flip(data, labx):
  images=tf.concat([data,
                    tf.image.flip_left_right(data),
                    tf.image.flip_up_down(data),
                    tf.image.flip_up_down(tf.image.flip_left_right(data))], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [10]:
def aug_zoom(data, labx):
  sz=np.shape(data)
  new=np.zeros((3,*sz))
  zoom=tf.image.resize(data, [int(sz[1]*3/4),int(sz[2]*3/4)])
  new[0,:,int(sz[1]/8):int(sz[1]*7/8),int(sz[2]/8):int(sz[2]*7/8),:]=zoom
  zoom=tf.image.resize(data, [int(sz[1]/2),int(sz[2]/2)])
  new[1,:,int(sz[1]/4):int(sz[1]*3/4),int(sz[2]/4):int(sz[2]*3/4),:]=zoom
  zoom=tf.image.resize(data, [int(sz[1]*3/2),int(sz[2]*3/2)])
  new[2,...]=zoom[:,int(sz[1]/4):int(sz[1]*5/4),int(sz[2]/4):int(sz[2]*5/4),:]
  images=tf.concat([data,
                    new[0],
                    new[1],
                    new[2]], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [11]:
def distortion_ima(data, sx=1, sy=3/2, ang=45):
  sz=np.shape(data)
  new=tfa.image.rotate(data,np.pi*ang/180)
  new=tf.image.resize(new, [int(sz[1]*sx),int(sz[2]*sy)])
  new=tfa.image.rotate(new, -np.pi*ang/180)
  szn=np.shape(new)
  new=new[:,int(np.abs(szn[1]-sz[1])/2):int((np.abs(szn[1]-sz[1])/2)+sz[1]),int(np.abs(szn[2]-sz[2])/2):int((np.abs(szn[2]-sz[2])/2)+sz[2]),:]
  return new

In [12]:
def aug_distortion(data, labx):
  images=tf.concat([data,
                    distortion_ima(data,1,3/2,45),
                    distortion_ima(data,3/2,1,45),
                    distortion_ima(data,3/2,1.1,120)], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [13]:
def rx(ix):
  r=np.random.choice(ix,ix,replace=False)
  return r

def aument(ind_0, tipo, aug=3):
  sz0=np.shape(ind_0)
  new0=np.zeros((aug,*sz0))

  for j in range(aug):
    new0[j,:,:int(sz0[1]/2),:int(sz0[2]/2),:]=ind_0[rx(sz0[0]),:int(sz0[1]/2),:int(sz0[2]/2),:]
    new0[j,:,:int(sz0[1]/2),int(sz0[2]/2):,:]=ind_0[rx(sz0[0]),:int(sz0[1]/2),int(sz0[2]/2):,:]
    new0[j,:,int(sz0[1]/2):,:int(sz0[2]/2),:]=ind_0[rx(sz0[0]),int(sz0[1]/2):,:int(sz0[2]/2),:]
    new0[j,:,int(sz0[1]/2):,int(sz0[2]/2):,:]=ind_0[rx(sz0[0]),int(sz0[1]/2):,int(sz0[2]/2):,:]
  
  ys=np.ones(aug*sz0[0])*tipo
  return new0, ys

In [14]:
def aug_cropping(data, labx):
  ind_0=np.array(data)[np.where(labx[:,0]==1)[0],...]
  ind_1=np.array(data)[np.where(labx[:,1]==1)[0],...]

  new0, lab0=aument(ind_0, 0)
  new1, lab1=aument(ind_1, 1)
  lab_0,_=one_hot(np.concatenate((lab0,lab1)))

  images=tf.concat([data, new0[0], new0[1], new0[2],
                    new1[0], new1[1], new1[2]], 0)
  lab=tf.concat([labx,lab_0], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [15]:
def sub_overlap(ind_0, tipo, aug=3):
  sz0=np.shape(ind_0)
  new0=np.zeros((aug,*sz0))

  for j in range(aug):
    new0[j]=ind_0[rx(sz0[0])]+ind_0[rx(sz0[0])]
    new0[j]=tf.image.per_image_standardization(new0[j])
    new0[j]=(new0[j]-np.min(new0[j]))/(np.max(new0[j])-np.min(new0[j]))  
  ys=np.ones(aug*sz0[0])*tipo
  return new0, ys

In [16]:
def aug_overlap(data, labx):
  ind_0=np.array(data)[np.where(labx[:,0]==1)[0],...]
  ind_1=np.array(data)[np.where(labx[:,1]==1)[0],...]

  new0, lab0=sub_overlap(ind_0, 0)
  new1, lab1=sub_overlap(ind_1, 1)
  lab_0,_=one_hot(np.concatenate((lab0,lab1)))

  images=tf.concat([data, new0[0], new0[1], new0[2],
                    new1[0], new1[1], new1[2]], 0)
  lab=tf.concat([labx,lab_0], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [17]:
def aug_add_noise(data, labx):
  sz1=np.shape(data)
  new=np.zeros((3,*sz1))
  for i in range(3):
    thres=0.05*(1+i)
    black=np.random.rand(*sz1)>thres
    white=np.random.rand(*sz1)<(1-thres)
    new[i]=(data*black*white)+(1-white)
  images=tf.concat([data, new[0], new[1], new[2]], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [18]:
def aug_color_space(data, labx):
  space1=tfio.experimental.color.rgb_to_ycbcr(np.array(data*255/np.max(data)).astype('uint8'))
  space2=tfio.experimental.color.rgb_to_hsv((np.array(space1)/np.max(space1)).astype('float32'))
  space3=tfio.experimental.color.xyz_to_rgb(space2)
  space1=np.array(space1).astype('float32')
  space1=(space1-np.min(space1))/(np.max(space1)-np.min(space1))
  images=tf.concat([data, space1, space2, space3], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [19]:
def aug_linear_kernel(data, labx):
  images=tf.concat([data,
                    tfa.image.gaussian_filter2d(data, (3,3), sigma=7),
                    tfa.image.gaussian_filter2d(data, (4,4), sigma=7),
                    tfa.image.gaussian_filter2d(data, (5,5), sigma=7)], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [20]:
def delete_box(data):
  sz1=np.shape(data)
  uno=np.zeros(sz1)
  xp=(np.random.rand(sz1[0])*sz1[1]).astype('int')
  yp=(np.random.rand(sz1[0])*sz1[2]).astype('int')
  uno[np.arange(sz1[0]),xp,yp,:]=1
  uno=tf.nn.dilation2d(uno, np.ones((int(sz1[1]/3),int(sz1[2]/3),3)), [1,1,1,1], 'SAME', 'NHWC', [1,1,1,1])
  uno=np.array(uno!=np.max(uno))
  new=data*uno
  return new

In [21]:
def aug_random_delete(data, labx):
  images=tf.concat([data,
                    delete_box(data),
                    delete_box(data),
                    delete_box(data)], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [22]:
from sklearn.decomposition import PCA

In [23]:
def noise(x,y,per=0.1):
  rand=np.random.rand(x,y)
  frac=(per)/(np.max(rand)-per)
  rand2=rand+frac
  rand2=rand2/np.max(rand2)
  return rand2

In [24]:
def pca_images(data, percentage=0.1):
  sz1=np.shape(data)
  flat=np.array(data[...,0]).reshape(sz1[0],sz1[1]*sz1[2])
  ncomp=np.min(np.shape(flat))
  pca = PCA(n_components=ncomp)
  pca.fit(flat)
  space1=pca.transform(flat)
  x,y=np.shape(space1)
  new=np.zeros((3,*sz1))
  for i in range(3):
    for j in range(3):
      rand=noise(x,y,percentage)
      ima=space1*rand
      ima=pca.inverse_transform(ima)
      ima=(ima-np.min(ima))/(np.max(ima)-np.min(ima)).astype('float32')
      ima=ima.reshape(sz1[0],sz1[1],sz1[2])
      new[i,:,:,:,j]=ima
  return new

In [25]:
def aug_PCA(data, labx):
  new=pca_images(data)
  images=tf.concat([data, new[0], new[1], new[2]], 0)
  lab=tf.concat([labx,labx,labx,labx], 0)
  sc=np.shape(images)[0]
  idr=np.random.choice(sc,sc,replace=False)
  images=tf.convert_to_tensor(np.array(images)[idr])
  lab=tf.convert_to_tensor(np.array(lab)[idr])
  return images, lab

In [26]:
method={'None': none_p,
        'Translation'	: aug_translation,
        'Rotation' : aug_rotation,
        'Flip' : aug_flip,
        'Resizing' : aug_zoom,
        'Distortion' : aug_distortion,
        'Cropping' : aug_cropping,
        'Overlapping' : aug_overlap,
        'Noise addition' : aug_add_noise,
        'Change of color space' : aug_color_space,
        'Linear filters' : aug_linear_kernel,
        'Random frame deletion' : aug_random_delete,
        'Based on PCA' : aug_PCA}

In [27]:
def get_data_training(indx, ima_tens, labels_v, k_fold='none', number_folds=10, augm='None'):
  if k_fold!='none':
    sz=len(indx)/number_folds
    valid=indx[int(k_fold*sz):int((k_fold+1)*sz)]
    train=np.concatenate((indx[:int(k_fold*sz)],indx[int((k_fold+1)*sz):])) 

    x_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[train]))
    v_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[valid]))
    y_data=tf.convert_to_tensor(np.array(labels_v)[train])
    vy_dat=tf.convert_to_tensor(np.array(labels_v)[valid])

    x_data, y_data=method[augm](x_data, y_data)
    v_data, vy_dat=method[augm](v_data, vy_dat)

    return x_data, v_data, y_data, vy_dat
  else:
    x_data=tf.image.grayscale_to_rgb(tf.convert_to_tensor(np.array(ima_tens)[indx]))
    y_data=tf.convert_to_tensor(np.array(labels_v)[indx])

    x_data, y_data=method[augm](x_data, y_data)

    return x_data, y_data

In [28]:
def reduce_data(labels, reduction=100):
  sub, clas=np.shape(labels)
  new_labels=[]
  for i in range(clas):
    i_tem=np.where(labels[:,i]==1)[0]
    rand=np.random.choice(np.shape(i_tem)[0], np.shape(i_tem)[0], replace=False)[:reduction]
    new_labels=np.append(new_labels,i_tem[rand])
  return new_labels.astype('int')

In [29]:
def get_data(sequences='T1'): #'FLAIR','T1_Gd'
  data_dir=('/content/drive/MyDrive/Brain_tumors_v2/Datasets/TCIA_LGG/x_x.mat')
  mats=sio.loadmat(data_dir.replace('x_x', sequences))
  images=mats['images']
  tumors=mats['size_tumor'][0]

  choise=np.argsort(np.var(np.var(images, axis=1), axis=1))[900:]
  images=images[choise]
  tumors=tumors[choise]

  images=images.reshape((np.shape(images)[0],np.shape(images)[1],np.shape(images)[2],1))
  labels=(tumors!=0).astype('int') #para TCIA tres canales pero la misma mascara
  images=tf.image.resize(images, [100,100], method='nearest')
  labels, n_class=one_hot(labels)
  indx=reduce_data(labels)
  return tf.convert_to_tensor(np.array(images)[indx]), labels[indx], n_class

In [30]:
from tensorflow.keras import applications as ap
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [31]:
def get_model(network, opt='adadelta', loss_name='categorical_crossentropy', input_shape=(240,240,3), classes=2, weights='imagenet'):
  try:
    del model
  except:
    print('done')
  model=Sequential()
  if network=='ResNet50V2':
    model.add(ap.ResNet50V2(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) # The input must have 3 channels
  if network=='EfficientNetB7':
    model.add(ap.EfficientNetB7(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='InceptionResNetV2':
    model.add(ap.InceptionResNetV2(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='InceptionV3':
    model.add(ap.InceptionV3(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='NASNetLarge':
    model.add(ap.NASNetLarge(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  if network=='VGG19':
    model.add(ap.VGG19(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes)) 
  if network=='Xception':
    model.add(ap.Xception(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  if network=='DenseNet121':
    model.add(ap.DenseNet121(include_top=False, weights=weights, input_shape=input_shape, pooling='avg', classes=classes))
  
  model.add(Dense(classes, activation='softmax'))
  model.compile(optimizer=opt, loss=loss_name, metrics=['acc', tf.keras.metrics.Recall(), tf.keras.metrics.FalsePositives()])
  return model

In [32]:
df = pd.DataFrame(columns=('run_n', 'k_fold', 'network', 'optimizer', 'loss', 'epochs', 'total_parameters', 'time', 'transfer', 'augm', 'Class', 'TP', 'TN', 'FP', 'FN','result_mat'))
df.head()

,run_n,k_fold,network,optimizer,loss,epochs,total_parameters,time,transfer,augm,Class,TP,TN,FP,FN,result_mat


In [33]:
TP=tf.keras.metrics.TruePositives()
TN=tf.keras.metrics.TrueNegatives()
FP=tf.keras.metrics.FalsePositives()
FN=tf.keras.metrics.FalseNegatives()

In [34]:
def run_experiment(images, labels, number_class, augm_method, name='', star_run=0, end_run=30, batch_size=4):
  for augm_con in augm_method: 
    for net in Networks:
      name_0='_'.join(['Augmentation', name, str(transferlearning), augm_con, net, optimizer])
      if not os.path.exists(path2+name_0+'.csv'):
        df.to_csv(path2+name_0+'.csv')

      train_id, test_id=get_index(labels)
      x_test, y_test=get_data_training(test_id, images, labels)
      y_test=y_test>=0.5

      for i in range(star_run, end_run): #Numero corridas 
        for j in [loss]: #Funciones de perdida
          cntn=True
          name_m='_'.join([name_0,j,'run',str(i)])  
          print(name_m)

          #Obtener imagenes nuevamente
          x_train, x_valid, y_train, y_valid=get_data_training(train_id, images, labels, k_fold=i%10, augm=augm_con)      
          model=get_model(net, classes=number_class, input_shape=list(np.shape(x_train)[1:]),  weights=transferlearning)
          
          try:
            tic = time.time()
            results = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=batch_size, epochs=epochs)
            toc=time.time()-tic
            model.save_weights(pathW+name_m+"w.h5")
          except:
            print('Training error')
            cntn=False

          if cntn:
            sio.savemat(pathW+name_m+'_r.mat', results.history)

            #Validation
            y_hat=np.array(model.predict(x_test))
            sio.savemat(pathW+name_m+'_los_8.mat',{'y_hat': y_hat, 'y_test': np.array(y_test)})
            y_hat=y_hat>=0.5

            for class_i in range(number_class):
              TP.reset_state()
              TN.reset_state()
              FP.reset_state()
              FN.reset_state()        

              TP.update_state(y_test[:,class_i], y_hat[:,class_i])
              TN.update_state(y_test[:,class_i], y_hat[:,class_i])
              FP.update_state(y_test[:,class_i], y_hat[:,class_i])
              FN.update_state(y_test[:,class_i], y_hat[:,class_i])
              total_p=model.count_params()

              #data frame
              df2=pd.read_csv(path2+name_0+'.csv')
              df2=df2.append({'run_n': i,
                              'k_fold': i%10,
                              'network': net,
                              'optimizer': optimizer,
                              'loss': 'categorical_crossentropy',
                              'epochs': epochs,
                              'total_parameters': total_p,
                              'time': toc,
                              'transfer': transferlearning,
                              'augm': augm_con,
                              'Class': class_i,
                              'TP': float(TP.result()),
                              'TN': float(TN.result()),
                              'FP': float(FP.result()),
                              'FN': float(FN.result()),
                              'result_mat': name_m+'_r.mat'} , ignore_index=True)
              df2=df2.drop(df2.columns[:np.where(df2.columns=='run_n')[0][0]], axis=1)
              df2.to_csv(path2+name_0+'.csv')
            del x_train, y_train, x_valid, y_valid, model
            clear_output(wait=True)
      star_run=0

# Run experiment

In [ ]:
Networks=['ResNet50V2', 'EfficientNetB7', 'InceptionResNetV2', 'InceptionV3', 'VGG19', 'Xception', 'DenseNet121']
path2='/content/drive/MyDrive/Brain_tumors_v2/Results/AumentoDeDatos/ceros/results_csv/'
pathW='/content/drive/MyDrive/Brain_tumors_v2/Results/AumentoDeDatos/ceros/Weights/'
transferlearning=None
optimizer='adadelta'
epochs=50
loss='categorical_crossentropy'
augmentation=['None', 'Translation', 'Rotation', 'Flip', 'Resizing', 'Distortion', 'Cropping', 'Overlapping', 'Noise addition', 'Change of color space', 'Linear filters', 'Random frame deletion', 'Based on PCA']


#----------------------------------RUN------------------------------------------
seq='FLAIR'#,'T1_Gd'
imas, labs, no_cls=get_data(seq)
run_experiment(imas, labs, no_cls, augmentation, seq, star_run=0)